In [1]:
import os
import flopy
import pyemu
import pandas as pd
import numpy as np
from pyemu.pst.pst_utils import SFMT

flopy is installed in C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\flopy


In [2]:
wd = "D:/Projects/Watersheds/Animas/Analysis/APEX-MODFLOWs/calibrations/qam_test/APEX-MODFLOW"
modwd = "D:/Projects/Watersheds/Animas/Analysis/APEX-MODFLOWs/calibrations/qam_test/APEX-MODFLOW/MODFLOW"
os.chdir(wd)
mname = "modflow.mfn"


### Model Setup

- Simulation Period 
    * Jan. 01, 1980 ~ Dec. 31, 1999 with 12 years warm-up period for 20 years
- Measurement Duration
    * Varies
    * Streamflow - Jan. 01, 1992 ~ Dec. 31, 2019
    * Watertable - 
    <br/><br/>
- Calibration / Validation
    * 1/1/1992 - 12/31/1999 for Calibration (8 years)
    * 2000 - 2019 for validation (20 years)
    <br/><br/>
- Streamflow: Little baseflow, High peak, Peak early -> , peak low, shift to little right
    - SWAT parameters:
        * Little Baseflow & High Peak
            - CN2 - Decrease
            - ESCO - Increase
            - SOL_AWC - Increase
        * Peak early
            * HRU_SLP - Decrease
            * OV_N - Increase
            * SLSUBBSN - Increase (The value of overland flow length)
    - MODFLOW parameters:
        * Little Baseflow & High Peak
            - Riverbed cond - Decrease
            - Riverbed bottom elevation - Decrease
        * K - ? 
        * Sy - ?
        
- Watertable -> high watertable, slow recession, 
    - SWAT parameters:
        * RCHRG_DP - (turned off) decrease
        * CN2 - ?
        * ESCO - ? (increase)
        * SOL_AWC - decrease
    - MODFLOW parameters:
        * K - increase 
        * Sy - ?
        * EVT depth - increase
        * River Bottom - ?
        * River conductance - decrease

In [3]:
from apexmf_pst_pkgs import apexmf_pst_utils

### 1. Create PEST input files (template / instruction)

#### 1.1. Create template files
We are going to use the *.pval and mf_river.par files for MODFLOW parameters and model.in file for SWAT parameters.

In [4]:
# pval file
pval_file = os.path.join(modwd, 'mf_1000.pval')
# parmfile
parm_file = 'PARM1501.DAT'

In [5]:
gw_par = pyemu.utils.gw_utils.modflow_pval_to_template_file(pval_file, tpl_file=None)
print(gw_par)

       parnme    parval1                       tpl
parnme                                            
hk01     hk01   0.018476   ~   hk01              ~
hk02     hk02  20.390900   ~   hk02              ~
hk03     hk03   0.001537   ~   hk03              ~
hk04     hk04   0.010009   ~   hk04              ~
hk05     hk05   0.808490   ~   hk05              ~
sy01     sy01   0.420007   ~   sy01              ~
sy02     sy02   0.017711   ~   sy02              ~
sy03     sy03   0.500000   ~   sy03              ~
sy04     sy04   0.500000   ~   sy04              ~
sy05     sy05   0.240478   ~   sy05              ~


In [5]:
# Create parm template file

sw_par = apexmf_pst_utils.parm_to_tpl_file()
sw_par

,0,1,2,3,4,5,6,7,8,9
0,.50000,2.5000,1.0E-5,1.0000,~p5 ~,.20000,.00320,1.0E-5,1.0000,7.0000
1,.10000,~p12 ~,.20000,2.0000,~p15 ~,~p16 ~,~p17 ~,1.500,0.003,~p20 ~
2,10.00,0.90,~p23 ~,0.1000,~p25 ~,0.000,0.300,5.000,.01,1.100
3,0.30,1.00,2.5000,0.,0.9900,0.,0.200,1.000,.01,~p40 ~
4,0.50,0.40,0.1500,2.0000,3.0000,0.500,1.000,0.000,~p49 ~,~p50 ~
5,~p51 ~,10.00,0.9000,0.6000,0.2000,0.900,1.000,0.600,0.100,7.000
6,~p61 ~,0.25,1.1000,1.0000,0.0010,3.000,10.000,0.500,0.500,1.
7,1.15,.05,0.5000,0.2000,0.8000,.05,0.000,10.000,1.000,.5
8,0.00,3.,0.5100,0.0001,0.0001,0.010,0.010,0.001,0.100,~p90 ~
9,~p91 ~,~p92 ~,1.,1.,7,1.,.9,1.,1.,.000548


## 1.2. Build instruction files (streamflow / watertable / baseflow)
### 1.2.1. Streamflow (output.rch)

In [4]:
# file path
rch_file = 'SITE75.RCH'
# reach numbers that are used for calibration
subs = [12, 57, 75]

In [5]:
# extract month_streamflow
apexmf_pst_utils.extract_month_str(rch_file, subs, '1/1/1980', '1/1/1992', '12/31/1999')

cha_012.txt file has been created...
cha_057.txt file has been created...
cha_075.txt file has been created...
Finished ...


In [6]:
apexmf_pst_utils.extract_month_sed(rch_file, subs, '1/1/1980', '1/1/1992', '12/31/1999')

sed_012.txt file has been created...
sed_057.txt file has been created...
sed_075.txt file has been created...
Finished ...


### 1.2.3. Create instruction files for each str_sim file using the 'streamflow.obd' file

In [4]:
apexmf_pst_utils.cvt_strobd_dtm()

In [7]:
# because we have 3 streamgages let's loop for them
# read streamobd and get column names
stf_obd = pd.read_csv(
                    'streamflow_month.obd',
                    sep='\t',
                    index_col=0,
                    parse_dates=True,
                    na_values=[-999, '']
                    )
# stf_obd_c = stf_obd.resample('M').mean()
# stf_obd_c.to_csv('streamflow_m.obd', sep='\t', na_rep=-999, float_format='%.2f')
obds = stf_obd.columns.tolist()
obds.remove('str_072')
obds.remove('sed_072')
print(obds)
sim_files = ['cha_{:03d}.txt'.format(x) for x in subs]
sed_files = ['sed_{:03d}.txt'.format(x) for x in subs]
sim_files = sim_files + sed_files
print(sim_files)

['str_012', 'str_057', 'str_075', 'sed_012', 'sed_057', 'sed_075']
['cha_012.txt', 'cha_057.txt', 'cha_075.txt', 'sed_012.txt', 'sed_057.txt', 'sed_075.txt']


In [8]:
# create instruction files for each sim file
for i in range(len(sim_files)):
    apexmf_pst_utils.stf_obd_to_ins(sim_files[i], obds[i], '1/1/1992', '12/31/1999', time_step='month')

cha_012.txt.ins file has been created...
cha_057.txt.ins file has been created...
cha_075.txt.ins file has been created...
sed_012.txt.ins file has been created...
sed_057.txt.ins file has been created...
sed_075.txt.ins file has been created...


In [4]:
# We do have watertable data now
grid_ids = [5895, 6273]
apexmf_pst_utils.extract_watertable_sim(grid_ids, '1/1/1980', '12/31/1999')



wt_5895.txt file has been created...
wt_6273.txt file has been created...


In [9]:
apexmf_pst_utils.mf_obd_to_ins('wt_5895.txt', 'gw_124', '1/1/1980', '12/31/1999')

wt_5895.txt.ins file has been created...


date
1980-01-01    l1
1980-01-02    l1
1980-01-03    l1
1980-01-04    l1
1980-01-05    l1
              ..
1999-12-27    l1
1999-12-28    l1
1999-12-29    l1
1999-12-30    l1
1999-12-31    l1
Name: gw_124_ins, Length: 7305, dtype: object

## Create a dummy pst file 

In [9]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'ani_dummy.pst')

# print(os.chdir(".."))
io_files

error using inschek for instruction file cha_012.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file cha_057.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file cha_075.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file sed_012.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file sed_057.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file sed_075.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file wt_5895.txt.ins:run() returned non-zero: 50
observations in this instru

(['mf_1000.pval.tpl', 'PARM1501.DAT.tpl'],
 ['mf_1000.pval', 'PARM1501.DAT'],
 ['cha_012.txt.ins',
  'cha_057.txt.ins',
  'cha_075.txt.ins',
  'sed_012.txt.ins',
  'sed_057.txt.ins',
  'sed_075.txt.ins',
  'wt_5895.txt.ins',
  'wt_6273.txt.ins'],
 ['cha_012.txt',
  'cha_057.txt',
  'cha_075.txt',
  'sed_012.txt',
  'sed_057.txt',
  'sed_075.txt',
  'wt_5895.txt',
  'wt_6273.txt'])

The ``parse_dir_for_io_files()`` helper is looking for files with the ".tpl" and ".ins" extension.  This assumes that the corresponding model input and model output files are the same name, minus the ".tpl" and ".ins" extension, respectively.  These file lists are then passed to another helper, which builds a basic control file for you (``Pst.from_io_files()``).  Let's look at this generic ``Pst`` instance:

In [10]:
par = pst.parameter_data
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
hk01,hk01,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
hk02,hk02,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
hk03,hk03,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
hk04,hk04,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
hk05,hk05,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p12,p12,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p15,p15,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p16,p16,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p17,p17,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1
p20,p20,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1


#### 2.1. Change parameter group name

In [11]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    # elif par.iloc[i, 0][:7] == 'rivbot_':
    #     par.iloc[i, 6] = 'rivbot'
    # elif par.iloc[i, 0][:6] == 'rivcd_':
    #     par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    elif par.iloc[i, 0][:1] == 'p':
        par.iloc[i, 6] = 'apex'
    # else:
    #     par.iloc[i, 6] = 'str'
print(par)

     parnme partrans parchglim  parval1       parlbnd       parubnd pargp  \
hk01   hk01      log    factor      1.0  1.100000e-10  1.100000e+10    hk   
hk02   hk02      log    factor      1.0  1.100000e-10  1.100000e+10    hk   
hk03   hk03      log    factor      1.0  1.100000e-10  1.100000e+10    hk   
hk04   hk04      log    factor      1.0  1.100000e-10  1.100000e+10    hk   
hk05   hk05      log    factor      1.0  1.100000e-10  1.100000e+10    hk   
p12     p12      log    factor      1.0  1.100000e-10  1.100000e+10  apex   
p15     p15      log    factor      1.0  1.100000e-10  1.100000e+10  apex   
p16     p16      log    factor      1.0  1.100000e-10  1.100000e+10  apex   
p17     p17      log    factor      1.0  1.100000e-10  1.100000e+10  apex   
p20     p20      log    factor      1.0  1.100000e-10  1.100000e+10  apex   
p23     p23      log    factor      1.0  1.100000e-10  1.100000e+10  apex   
p25     p25      log    factor      1.0  1.100000e-10  1.100000e+10  apex   

# 2.2. Set par ranges and initial values for parameters

### 2.2.3. MODFLOW

### Let's start values from 1st round of calibration result

In [12]:
# for MODFLOW parameters
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk01'):  # hk_sed
        par.iloc[i, 3] = 0.001    
        par.iloc[i, 4] = 0.001 / 100
        par.iloc[i, 5] = 0.001 * 1000
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk02'):  # hk_as01
        par.iloc[i, 3] = 26.1   
        par.iloc[i, 4] = 26.1 / 100
        par.iloc[i, 5] = 26.1 * 100
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk03'):  # hk_as02
        par.iloc[i, 3] = 0.01   
        par.iloc[i, 4] = 0.01 / 100
        par.iloc[i, 5] = 0.01 * 1000
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk04'):  # hk_as03
        par.iloc[i, 3] = 0.3   
        par.iloc[i, 4] = 0.3 / 100
        par.iloc[i, 5] = 0.3 * 1000  
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk05'):  # hk_org
        par.iloc[i, 3] = 11   
        par.iloc[i, 4] = 11 / 100
        par.iloc[i, 5] = 11 * 100  
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy01'):  # sy_sed
        par.iloc[i, 3] = 0.1      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy02'):  # sy_as01
        par.iloc[i, 3] = 0.1      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy03'):  # sy_as02
        par.iloc[i, 3] = 0.1      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01  
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy04'):  # sy_as03
        par.iloc[i, 3] = 0.1      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01 
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy05'):  # sy_org
        par.iloc[i, 3] = 0.1      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    else:
        count += 1
print(count)

16


In [13]:
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
hk01,hk01,log,factor,0.001,1.000000e-05,1.000000e+00,hk,1.0,0.0,1
hk02,hk02,log,factor,26.100,2.610000e-01,2.610000e+03,hk,1.0,0.0,1
hk03,hk03,log,factor,0.010,1.000000e-04,1.000000e+01,hk,1.0,0.0,1
hk04,hk04,log,factor,0.300,3.000000e-03,3.000000e+02,hk,1.0,0.0,1
hk05,hk05,log,factor,11.000,1.100000e-01,1.100000e+03,hk,1.0,0.0,1
p12,p12,log,factor,1.000,1.100000e-10,1.100000e+10,apex,1.0,0.0,1
p15,p15,log,factor,1.000,1.100000e-10,1.100000e+10,apex,1.0,0.0,1
p16,p16,log,factor,1.000,1.100000e-10,1.100000e+10,apex,1.0,0.0,1
p17,p17,log,factor,1.000,1.100000e-10,1.100000e+10,apex,1.0,0.0,1
p20,p20,log,factor,1.000,1.100000e-10,1.100000e+10,apex,1.0,0.0,1


# APEX

In [14]:
pst.parameter_data = apexmf_pst_utils.export_pardb_pest(par)


In [15]:
par = pst.parameter_data
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
hk01,hk01,log,factor,0.00100,0.00001,1.0000,hk,1.0,0.0,1
hk02,hk02,log,factor,26.10000,0.26100,2610.0000,hk,1.0,0.0,1
hk03,hk03,log,factor,0.01000,0.00010,10.0000,hk,1.0,0.0,1
hk04,hk04,log,factor,0.30000,0.00300,300.0000,hk,1.0,0.0,1
hk05,hk05,log,factor,11.00000,0.11000,1100.0000,hk,1.0,0.0,1
p12,p12,log,factor,2.50000,1.50000,2.5000,apex,1.0,0.0,1
p15,p15,log,factor,0.00001,0.00001,0.3000,apex,1.0,0.0,1
p16,p16,log,factor,1.00000,1.00000,1.5000,apex,1.0,0.0,1
p17,p17,log,factor,0.25000,0.00001,0.5000,apex,1.0,0.0,1
p20,p20,log,factor,0.20000,0.05000,0.4000,apex,1.0,0.0,1


# - River Parameters

In [29]:
# Let's use unfchg for riv_bot
# set +- 100 meters to default ranges and 0.001 for initials
for i in range(len(par)):
    if (par.iloc[i, 6]) == 'rivbot':  # rivbot 
        par.iloc[i, 3] = 30.1   # initial    
        par.iloc[i, 4] = 0.1   # lower
        par.iloc[i, 5] = 60   # upper
        par.iloc[i, 8] = -30   # offset

# Distiguish channels between high and low baseflow
# for low baseflow, set -500 min, 100 max, and -300 for initals
# lowbases = ['g240']
# for i in lowbases:
#     par.loc['rivbot_{}'.format(i), 'parval1'] = 600.1
#     par.loc['rivbot_{}'.format(i), 'parlbnd'] = 100
#     par.loc['rivbot_{}'.format(i), 'parubnd'] = 700
#     par.loc['rivbot_{}'.format(i), 'offset'] = -600
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
awc_147,awc_147,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_227,awc_227,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_240,awc_240,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_243,awc_243,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_66,awc_66,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
...,...,...,...,...,...,...,...,...,...,...
sy04,sy04,log,factor,0.404763,0.001,0.5,sy,1.0,0.0,1
sy05,sy05,log,factor,0.028398,0.001,0.5,sy,1.0,0.0,1
sy06,sy06,log,factor,0.197691,0.001,0.5,sy,1.0,0.0,1
sy07,sy07,log,factor,0.446098,0.001,0.5,sy,1.0,0.0,1


In [30]:
# Let's use pctchg for rivcd
# set +- 50 % to default ranges and 0.001 for initials
for i in range(len(par)):
    if (par.iloc[i, 6]) == 'rivcd':  # rivbot 
        par.iloc[i, 3] = 50.01   # initial    
        par.iloc[i, 4] = 0.1   # lower
        par.iloc[i, 5] = 100   # upper
        par.iloc[i, 8] = -50   # offset
# decrease in peak and increase in baseflow can be accomplished by decreasing river conductance
# for H peaks and L bases, set inital -20, max 10, min -50
# HpLb = [124, 92, 147, 66, 138, 228, 79]
# for i in lowbases:
#     par.loc['rivcd_{}'.format(i), 'parval1'] = 50.001
#     par.loc['rivcd_{}'.format(i), 'parlbnd'] = 0.001
#     par.loc['rivcd_{}'.format(i), 'parubnd'] = 60
#     par.loc['rivcd_{}'.format(i), 'offset'] = -50
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
awc_147,awc_147,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_227,awc_227,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_240,awc_240,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_243,awc_243,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
awc_66,awc_66,log,factor,1.001000,0.100,1.9,str,1.0,-1.0,1
...,...,...,...,...,...,...,...,...,...,...
sy04,sy04,log,factor,0.404763,0.001,0.5,sy,1.0,0.0,1
sy05,sy05,log,factor,0.028398,0.001,0.5,sy,1.0,0.0,1
sy06,sy06,log,factor,0.197691,0.001,0.5,sy,1.0,0.0,1
sy07,sy07,log,factor,0.446098,0.001,0.5,sy,1.0,0.0,1


## Observation

In [16]:
# set observation group
obd = pst.observation_data
obd

,obsnme,obsval,weight,obgnme
gw_098_19950510,gw_098_19950510,1.000000e+10,1.0,obgnme
gw_098_19950626,gw_098_19950626,1.000000e+10,1.0,obgnme
gw_098_19950725,gw_098_19950725,1.000000e+10,1.0,obgnme
gw_098_19951027,gw_098_19951027,1.000000e+10,1.0,obgnme
gw_098_19960213,gw_098_19960213,1.000000e+10,1.0,obgnme
...,...,...,...,...
str_075_199908,str_075_199908,1.000000e+10,1.0,obgnme
str_075_199909,str_075_199909,1.000000e+10,1.0,obgnme
str_075_199910,str_075_199910,1.000000e+10,1.0,obgnme
str_075_199911,str_075_199911,1.000000e+10,1.0,obgnme


In [17]:
# Change obd group name
for i in range(len(obd)):
    if obd.iloc[i, 0][:2] == 'gw':
        obd.iloc[i, 3] = obd.iloc[i, 0][:6]
    else:
        obd.iloc[i, 3] = obd.iloc[i, 0][:-7]
obd

,obsnme,obsval,weight,obgnme
gw_098_19950510,gw_098_19950510,1.000000e+10,1.0,gw_098
gw_098_19950626,gw_098_19950626,1.000000e+10,1.0,gw_098
gw_098_19950725,gw_098_19950725,1.000000e+10,1.0,gw_098
gw_098_19951027,gw_098_19951027,1.000000e+10,1.0,gw_098
gw_098_19960213,gw_098_19960213,1.000000e+10,1.0,gw_098
...,...,...,...,...
str_075_199908,str_075_199908,1.000000e+10,1.0,str_075
str_075_199909,str_075_199909,1.000000e+10,1.0,str_075
str_075_199910,str_075_199910,1.000000e+10,1.0,str_075
str_075_199911,str_075_199911,1.000000e+10,1.0,str_075


## 2.3. Import measured data

In [18]:
gwt_obd = pd.read_csv('MODFLOW/modflow.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
gwt_obd = gwt_obd['1/1/1992': '12/31/1999']
gwt_obd = gwt_obd[['gw_098', 'gw_124']]
gwt_obd

,gw_098,gw_124
date,,
1992-01-01,NaN,NaN
1992-01-02,NaN,NaN
1992-01-03,NaN,NaN
1992-01-04,NaN,NaN
1992-01-05,NaN,NaN
...,...,...
1999-12-27,NaN,NaN
1999-12-28,NaN,NaN
1999-12-29,NaN,NaN


In [19]:
stf_obd = pd.read_csv('streamflow_month.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd['1/1/1992': '12/31/1999']
stf_obd = stf_obd.drop(['str_072', 'sed_072'], axis=1)
stf_obd




,str_012,str_057,str_075,sed_012,sed_057,sed_075
date,,,,,,
1992-01-31,1.14,8.10,8.24,0.78,27.87,28.73
1992-02-29,1.30,7.98,9.07,1.00,27.13,34.35
1992-03-31,1.69,12.86,12.74,1.61,65.28,64.75
1992-04-30,5.10,40.43,30.79,14.72,579.21,335.07
1992-05-31,18.74,87.18,71.35,133.03,2168.35,1515.66
...,...,...,...,...,...,...
1999-08-31,14.36,67.16,73.09,79.53,1341.29,1572.21
1999-09-30,9.51,39.46,38.12,40.61,582.53,586.01
1999-10-31,4.04,12.97,10.27,8.00,68.33,44.99


In [20]:
# Get sub list based on obd order
sub_order = []
for i in obd.obgnme.tolist():
    if i not in sub_order:
        sub_order.append(i)
sub_order

['gw_098',
 'gw_124',
 'sed_012',
 'sed_057',
 'sed_075',
 'str_012',
 'str_057',
 'str_075']

In [21]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in sub_order[:2]:
    tot_obd += gwt_obd[i].dropna().tolist()
for j in sub_order[2:]:
    tot_obd += stf_obd[j].dropna().tolist()    
len(tot_obd)
# tot_obd

630

In [22]:
obd.loc[:, 'obsval'] = tot_obd
obd

,obsnme,obsval,weight,obgnme
gw_098_19950510,gw_098_19950510,-31.09,1.0,gw_098
gw_098_19950626,gw_098_19950626,-30.30,1.0,gw_098
gw_098_19950725,gw_098_19950725,-30.14,1.0,gw_098
gw_098_19951027,gw_098_19951027,-29.92,1.0,gw_098
gw_098_19960213,gw_098_19960213,-30.37,1.0,gw_098
...,...,...,...,...
str_075_199908,str_075_199908,73.09,1.0,str_075
str_075_199909,str_075_199909,38.12,1.0,str_075
str_075_199910,str_075_199910,10.27,1.0,str_075
str_075_199911,str_075_199911,7.72,1.0,str_075


In [23]:
# Change obd group name
for i in range(len(obd)):
    if obd.iloc[i, 3][:3] == "sed":
        obd.iloc[i, 2] = 0
obd

,obsnme,obsval,weight,obgnme
gw_098_19950510,gw_098_19950510,-31.09,1.0,gw_098
gw_098_19950626,gw_098_19950626,-30.30,1.0,gw_098
gw_098_19950725,gw_098_19950725,-30.14,1.0,gw_098
gw_098_19951027,gw_098_19951027,-29.92,1.0,gw_098
gw_098_19960213,gw_098_19960213,-30.37,1.0,gw_098
...,...,...,...,...
str_075_199908,str_075_199908,73.09,1.0,str_075
str_075_199909,str_075_199909,38.12,1.0,str_075
str_075_199910,str_075_199910,10.27,1.0,str_075
str_075_199911,str_075_199911,7.72,1.0,str_075


### 4. Export control file

In [24]:
pst.control_data.noptmax=0

In [25]:
pst.model_command = 'python forward_run.py'

In [26]:
pst.write('ani_pest.pst')

noptmax:0, npar_adj:26, nnz_obs:342


In [27]:
obd

,obsnme,obsval,weight,obgnme
gw_098_19950510,gw_098_19950510,-31.09,1.0,gw_098
gw_098_19950626,gw_098_19950626,-30.30,1.0,gw_098
gw_098_19950725,gw_098_19950725,-30.14,1.0,gw_098
gw_098_19951027,gw_098_19951027,-29.92,1.0,gw_098
gw_098_19960213,gw_098_19960213,-30.37,1.0,gw_098
...,...,...,...,...
str_075_199908,str_075_199908,73.09,1.0,str_075
str_075_199909,str_075_199909,38.12,1.0,str_075
str_075_199910,str_075_199910,10.27,1.0,str_075
str_075_199911,str_075_199911,7.72,1.0,str_075
